# Mercado Profesional de Swaps

- Este mercado también se conoce como mercado OTC (over-the-counter) de swaps.


- En el notebook anterior vimos la estructura de los contratos que se operan en el mercado profesional.


- La otra característica fundamental de este mercado es el uso de [colateral](https://www.youtube.com/watch?v=9K14BBbIxXM).
  - **Margen inicial:** cantidad de dinero up-front antes de iniciar a operar. No siempre es > 0.
  - **Margen de variación:** cantidad que coincide con la variación en el valor **de la cartera** y que es entregada por la contraparte que sufre la pérdida.
  - Importante que se considere el valor de toda la cartera (todas las operaciones de derivados entre 2 contrapartes). Esto se sustenta en el llamado [**close-out netting**](https://www.youtube.com/watch?v=aa3TdhvoOSA).
  - **Close-out netting:** acuerdo para compensar, en caso de quiebra, las cuentas por cobrar y recibir de una cartera de derivados entre dos contrapartes. La legislación aplicable al contrato debe permitir este mecanismo.

- Existen dos formas de implementar el uso de colateral:
  - Contrato bilateral de condiciones generales y un anexo de mitigación de riesgo de crédito. Por ejemplo el contrato ISDA y su Credit Support Annex (CSA). Las principales cláusulas negociadas en un CSA son:
    - **Threshold:** define el nivel de exposición (mark-to-market) bajo la cual no es necesario entregar colateral. El threshold puede ser distinto para cada contraparte y típicamente se define en función del rating crediticio.
    - **Colateral Elegible:** define el tipo de instrumentos que pueden ser entregados como colateral. Lo más usual es cash, a veces en más de una moneda. También se pueden aceptar instrumentos de renta fija de bajo riesgo valorizados utilizando un *haircut* (descuento por sobre el valor de mercado del instrumento).
    - **Minimum transfer amount (MTA):** corresponde a la cantidad por la cual la exposición de una contraparte debe exceder el monto de colateral ya entregado para solicitar colateral adicional.
    - **Remuneración del colateral:** el colateral entregado en cash gana intereses. Aquí se define qué índice de tasa de interés se utiliza para calcular esos intereses (típicamente un índice overnight). Cuando el colateral se entrega en forma de instrumentos de renta fija, quien entrega tiene derecho a recibir los cupones del instrumento.
    - **Periodicidad de Cálculo:** con qué periodicidad se calcula el valor de la cartera y se determina si corresponde exigir o devolver colateral.
  - Utilizar [una contraparte central (CCP) de derivados](https://www.youtube.com/watch?v=o-Yxjs33aQs).
    - La regulación de EE. UU. y de Europa requiere que los bancos más grandes y que operan globalmente lleven sus transacciones de derivados a una CCP.
    - En Chile existe [Comder](https://www.comder.cl), aunque no es obligatorio para los bancos utilizarla.

## Ejemplo

Consideremos el contrato definido por el siguiente payoff.

In [1]:
def payoff(nominal, tasa_fija, tasa_flotante, plazo):
    """
    Forward de tasa de interés.
    """
    return nominal * (tasa_fija - tasa_flotante) * plazo / 360

Consideremos los siguientes parámetros y comparemos los flujos de caja en el caso que el contrato se opere con o sin colateral.

In [2]:
# Plazo en días hasta el vencimiento del contrato
plazo_a_vcto = 30

# Nominal o nocional del contrato
nominal = 10000000

# Tasa fija o strike del contrato
tasa_fija = .005

# Plazo de la tasa del contrato
plazo = 92

# Escenarios (valores posibles de la tasa flotante al vencimiento)
escenarios = [.004, .006]

# Tasa de financiamiento del banco
tasa_fin = .0055

# Tasa de remuneración del colateral
tasa_on = .0045

print(f'nominal: {nominal:,.0f}')

nominal: 10,000,000


### Escenario 1

Flujos de caja sin colateral en el primer escenario, que suponemos se produce instantáneamente (o sea la tasa variable cambia justo después de celebrar el contrato).

In [3]:
payoff_esc_0 = payoff(nominal, tasa_fija, escenarios[0], plazo)
print(f"Flujo de caja sin colateral: {payoff_esc_0:,.4f}")

Flujo de caja sin colateral: 2,555.5556


Justo después de celebrar el contrato, el valor del contrato cambia (se supone por simplicidad que coincide con el valor del payoff):

In [4]:
m2m_esc_0 = payoff(nominal, tasa_fija, escenarios[0], plazo)
print(f'Valor de mercado: {m2m_esc_0:,.4f}')

Valor de mercado: 2,555.5556


Como el valor es positivo, lo recibo como colateral, esto ocasiona un costo, porque debo remunerar el colateral, y un beneficio, porque recibo una remuneración por ese colateral si lo reinvierto a mi tasa de financiamiento (esto último, sólo en el caso que la reinversión del colateral esté permitida en el CSA).

In [5]:
costo_esc_0 = -m2m_esc_0 * tasa_on * plazo_a_vcto / 360
print(f'Costo: {costo_esc_0:,.4f}')

Costo: -0.9583


In [6]:
beneficio_esc_0 = m2m_esc_0 * tasa_fin * plazo_a_vcto / 360
print(f'Beneficio: {beneficio_esc_0:,.4f}')

Beneficio: 1.1713


Por lo tanto, el flujo total en este caso:

In [7]:
flujo_total_esc_0 = payoff_esc_0 + beneficio_esc_0 + costo_esc_0
print(f'Flujo total: {flujo_total_esc_0:,.4f}')

Flujo total: 2,555.7685


### Escenario 2

En el caso en que la tasa sube el flujo sin colateral es:

In [8]:
payoff_esc_1 = payoff(nominal, tasa_fija, escenarios[1], plazo)
print(f"Flujo de caja sin colateral: {payoff_esc_1:,.4f}")

Flujo de caja sin colateral: -2,555.5556


El valor de mercado del contrato cambia y es igual a:

In [9]:
m2m_esc_1 = payoff(nominal, tasa_fija, escenarios[1], plazo)
print(f'Valor de mercado: {m2m_esc_1:,.4f}')

Valor de mercado: -2,555.5556


Como el valor es negativo, tengo que entregarlo como colateral, esto ocasiona un costo, porque debo financiar el colateral, y un beneficio, porque recibo una remuneración por ese colateral.

In [10]:
costo_esc_1 = m2m_esc_1 * tasa_fin * plazo_a_vcto / 360
print(f'Costo: {costo_esc_1:,.4f}')

Costo: -1.1713


In [11]:
beneficio_esc_1 = -m2m_esc_1 * tasa_on * plazo_a_vcto / 360
print(f'Beneficio: {beneficio_esc_1:,.4f}')

Beneficio: 0.9583


El flujo total en este caso es:

In [12]:
flujo_total_esc_1 = payoff_esc_1 + beneficio_esc_1 + costo_esc_1
print(f'Flujo total: {flujo_total_esc_1:,.4f}')

Flujo total: -2,555.7685


### Observaciones

- El uso de colateral debe considerarse al momento de hacer el pricing de una operación.


- El efecto en pricing depende del costo de financiamiento del banco.